# CVPUD dataset
## Context
## Regression Tree
## Dataset preparation

In [1]:
# Import necessary libraries
from encodings import search_function
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Import necessary tools from the sklearn library
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, plot_confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, KFold

# Import sklearn library tools used ONLY for validating my results
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve, f1_score, accuracy_score, recall_score, precision_score, auc, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.decomposition import PCA
from scipy.stats import randint as sp_randint
import warnings
warnings.filterwarnings('ignore')

In [3]:
# load datasets function
def load_data(data_file_name):
    data_dir = "..\..\..\data\data_regression"
    data_path = os.path.join(data_dir, data_file_name)
    df = pd.read_csv(data_path)
    data_X = df.iloc[:,:-1]
    data_y = df.iloc[:,-1]
    scaler_X = StandardScaler()
    data_X = scaler_X.fit_transform(data_X)
    data_y = pd.Categorical(data_y).codes.reshape(-1)
    return data_X, data_y

In [4]:
def main():
    
    # read dataset from csv file
    data_name = "CVPUD_regression"
    data_X, data_y = load_data("{}.csv".format(data_name))
    data_X, data_y = pd.DataFrame(data_X), pd.Series(data_y)
    data_y = (data_y - data_y.mean()) / data_y.std()

    # Randomly assingning a train and test set
    train_X, test_X, train_y, test_y = train_test_split(data_X, data_y, test_size=0.33, random_state=2200)
    return train_X, test_X, train_y, test_y, data_X, data_y

In [19]:
# I am loading the full dataset and renaming the columns to keep better track of each attribute
data_dir = "..\..\..\data\data_regression"
data_path = os.path.join(data_dir, "CVPUD_regression.csv")
df = pd.read_csv(data_path, header=0)
df.head()

,subject#,age,sex,test_time,motor_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,...,Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE,total_UPDRS
0,1,72,0,5.6431,28.199,0.00662,0.000034,0.00401,0.00317,0.01204,...,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006,34.398
1,1,72,0,12.6660,28.447,0.00300,0.000017,0.00132,0.00150,0.00395,...,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810,34.894
2,1,72,0,19.6810,28.695,0.00481,0.000025,0.00205,0.00208,0.00616,...,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014,35.389
3,1,72,0,25.6470,28.905,0.00528,0.000027,0.00191,0.00264,0.00573,...,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277,35.810
4,1,72,0,33.6420,29.187,0.00335,0.000020,0.00093,0.00130,0.00278,...,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361,36.375


# 1.2 Implement the class of regression tree

In [6]:
### criterion function for regression tree
def sum_squared_distance_to_mean(X, y):
    '''
    function used to calculate the squared error.
    '''
    n = X.shape[0]
    return np.var(y) * n

class DecisionTreeRegressor(object):
    '''
    This class is for decision tree regression

    Attributes:
        - criterion: a function used as the criterion of decision tree regression
        - tolS: a tolerance on the error reduction. we will stop the splitting if
            low error reduction.
        - tree: a nested dictionary representing the decision tree structure.
    '''
    def __init__(self, criterion, tolS=0.1):
        # Initialization
        self.criterion = criterion
        self.tolS = tolS

    def fit(self, X, y):
        '''
        function used to fit the decision tree regressor

        Args:
            X - features of training samples, a pandas dataframe with shape (n, d), where
                X.columns is the column name of X, and we can use X['feat'] to index all
                values of the feature named `feat`.
            y - target values (continuous, scaled) of training samples, a pandas series with shape (n,)
        '''
        self.tree = self.create_tree(X, y)

    def predict(self, X):
        '''
        function used to fit the decision tree regressor

        Args:
            X - features of test samples, a pandas dataframe with shape (n, d)

        Returns:
            y - predictions of test samples, a pandas series with shape (n,)
        '''
        n = X.shape[0]
        y = []
        for i in range(n):
            y.append(self.predict_each(X.iloc[i], self.tree))
        y = pd.Series(y)
        y.index = X.index
        return y

    @staticmethod
    def choose_best_split(X, y, criterion, tolS=1):
        '''
        function used to choose the best split feature and split point

        Args:
            X - features of test samples, a pandas dataframe with shape (n, d)
            y - target values of training samples, a pandas series with shape (n,)
            criterion - function used to measure the quality of a split
            tolS - a tolerance on the error reduction.

        Returns:
            best_feat: the feature used to split on for this node
            best_split: the value of the feature used to split for this node
        '''
        # used to calculate the error reduction
        origin_score = criterion(X, y)
        # initialize
        best_feat, best_split = None, None
        best_score = np.inf
        # search for each feature
        for feat in X.columns:
            # if all values of this feature are equal, do not split this feature
            X_feat_value = set(X[feat])
            if len(X_feat_value) == 1:
                continue
            # otherwise, search for each possible split point of this feature
            for split in X_feat_value:
                # divide the dataset into two parts according to the split
                idx1 = X[feat] < split
                idx2 = X[feat] >= split
                # calculate score to evaluate the quality of a split
                score1 = criterion(X.loc[idx1], y.loc[idx1])
                score2 = criterion(X.loc[idx2], y.loc[idx2])
                score = score1 + score2
                if score < best_score:
                    # choose the split with the largest (variance) reduction
                    best_feat = feat
                    best_split = split
                    best_score = score
        if origin_score - best_score < tolS:
            # Do not split if the (variance) reduction is low
            return None, None
        else:
            # return the feature and the value used for the split
            return best_feat, best_split

    def create_tree(self, X, y):
        '''
        build the decision tree regressor in a recursive manner.
        use a dictionary to represent tree node:
            if the tree node is an internal node, the dictionary will have the following five items:
                - tree["is_leaf"] stores whether the tree node is a leaf node or not.
                - tree["split_feat"] stores the feature used to split on for this node
                - tree["split_point"] stores the value of the feature used to split
                - tree["left"] is a (nested) dictionary used to store the left subtree of this node.
                - tree["right"] is a (nested) dictionary used to store the right subtree of this node.
            if the tree node is a leaf node, the dictionary will have the following two items:
                - tree["is_leaf"] stores whether the tree node is a leaf node or not.
                - tree["value"] stores the prediction at the leaf node
        returns a nested dictionary used to store the tree structure.
        '''
        Tree = {}
        # create a leaf node if all values are equal
        y_values, y_counts = np.unique(y, return_counts=True)
        if len(y_counts) == 1:
            Tree["is_leaf"] = True
            Tree["value"] = y_values[0]
            return Tree
        # create a leaf node if feature set is empty (a low error reduction)
        feat, split = self.choose_best_split(X, y, self.criterion, self.tolS)
        if feat is None:
            Tree["is_leaf"] = True
            Tree["value"] = y.mean()
            return Tree
        # otherwise, create an internal node
        Tree["is_leaf"] = False
        Tree["split_feat"] = feat
        Tree["split_point"] = split
        # divide the dataset (X, y) into two parts according to tree split
        # build the left subtree
        idx = X[feat] < split
        Tree["left"] = self.create_tree(X.loc[idx], y.loc[idx])
        # build the right subtree
        idx = X[feat] >= split
        Tree["right"] = self.create_tree(X.loc[idx], y.loc[idx])
        return Tree

    @staticmethod
    def predict_each(x, tree):
        '''
        for each sample, get the prediction of decision tree regressor in a recursive manner.

        Args:
            x - features of a sample, a pandas Series with shape (d,)
            tree - a nested dictionary representing the decision tree structure.

        Returns:
            the prediction of the sample `x`
        '''
        if tree["is_leaf"] is True:
            # if the `tree` is a leaf node, get the prediction at the leaf node
            return tree["value"]
        else:
            # the 'tree' is a nested dictionary with the following four items:
            #     `split_feat`, `split_point`, `left`, 'right`.
            # get the feature used to split on for this tree
            feat = tree["split_feat"]
            # get the value of the feature used to split
            split = tree["split_point"]
            # get the value of the feature for the sample `x`
            value = x[feat]
            if value < split:
                # search for the left subtree
                return DecisionTreeRegressor.predict_each(x, tree["left"])
            else:
                # search for the right subtree
                return DecisionTreeRegressor.predict_each(x, tree["right"])


## 1.3 Fit the model 

In [7]:
def run():
    # dataset preparation
    X, y = main()[0], main()[2]
    # initialize the decision tree regressor
    model = DecisionTreeRegressor(criterion=sum_squared_distance_to_mean, tolS=0.1)
    # fit the regression tree
    model.fit(X, y)
    # get the prediction of samples
    y_hat = model.predict(main()[1])
    # calculate the mean squared error of samples
    print("The MSE of Random Regression Tree is:", ((y - y_hat)**2).mean())

In [8]:
run()

The MSE of Random Regression Tree is: nan


# Suppor Vector Regressor

In [12]:
# I am loading the full dataset and renaming the columns to keep better track of each attribute
data_dir = "..\..\..\data\data_regression"
data_path = os.path.join(data_dir, "CVPUD_regression.csv")
df = pd.read_csv(data_path, header=0)
df.head()
df.shape
X = np.array(df.iloc[:,:-1])[:2214,:]
y = np.array(df.iloc[:,-1])[:2214]
# normalize
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

# split the train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2022)

## 2.2 Implementation of SVR

In [13]:
class SVR():
    '''
    This is a class for support vector regressor.
    
    The class contains the hyper-parameters such as $C$, $\epsilon$ and the kernel bandwidth $\sigma$. It also contains 
    the alpha vector, the tolerance for prediction error and the maximum number of iteration.
    
    It contains the functions for calculating the kernel matrix, fitting the model to estimate alpha, hat_alpha and b 
    with SMO algorithm, making predictions and other fundamental functions.
    
    Attributes:
        C (positive number)         - the hyperparameter for SVM algorithm
        sigma (positive number)     - the kernel bandwidth $\sigma$ of Gaussian kernel 
        toler (positive number)     - $\epsilon$, the threshold value of prediction error. If the prediction error of 
                                      a sample is larger than this value, the corresponding alphas[i] or hat_alpha[i]
                                      will be probably updated.
        maxIter (positive integer)  - the maximum number of iteration to search a pair of alpha's to update
        alphas (vector, num_samples) - the alpha vector in the dual problem 
        hat_alphas (vector, num_samples) - the hat_alpha vector in the dual problem 
        b (number)                  - the bias b
    '''
    
    def __init__(self, C = 1, sigma = 1, toler = 1, maxIter = 10):
        self.C = C
        self.sigma = sigma
        self.toler = toler
        self.maxIter = maxIter
        self.alphas = 0
        self.hat_alphas = 0
        self.b = 0
        
    def rbfkernel(self, X, Y):
        '''
        Calculate the kernel matrix whose (i,j)-th entry is $k(X[i,:], Y[j,:])$.
        '''
        m = X.shape[0]
        n = Y.shape[0]
        K = np.zeros(shape=(m, n))
        for i in range(m):
            for j in range(n):
                K[i, j] = np.sum((X[i,:] - Y[j,:])**2)
        K = np.exp(-K / (2 * self.sigma**2))
        return K

    def selectJrand(self, i, m):
        '''
        Randomly choose an index $j\neq i$ from 0 to m-1 and choose alpha[i] (or hat_alpha[i]) if hat_alpha[i] = 0 (or alpha[i]=0).
        '''
        j = i
        while (j == i):
            j = int(np.random.uniform(0, m))
        if self.alphas[j] == 0:
            choose_hat_j = 1
        else:
            choose_hat_j = 0
        return (j, choose_hat_j)


    def clipAlpha(self, aj, H, L):
        '''
        Clip the vale aj by the lower bound L and upper bound H
        '''
        if aj > H:
            aj = H
        if L > aj:
            aj = L
        return aj


    def fit(self, X_train, y_train):
        '''
        estimate the alphas vector, hat_alpha vector and bias in the SVM model
        
        Args: 
            X_train (matrix, num_train*num_features): features of training samples
            y_train (vector, num_train): output of training samples
            
        Returns:
            self.b (a number)                     : the bias
            self.alphas (vector, num_samples)     : the alpha vector 
            self.hat_alphas (vector, num_samples) : the hat_alpha vector 
        ''' 
        K_train = self.rbfkernel(X_train, X_train)
        self.X_train = X_train
        m = K_train.shape[0]
        self.alphas = np.zeros((m, ))
        self.hat_alphas = np.zeros((m, ))
        num_iter = 0
        while (num_iter < self.maxIter):
            alphaPairsChanged = 0
            # optimize for each data vector (with kernel trick)
            for i in range(m):  
                ### choose hat_alphas[i] or alphas[i] to update
                if self.hat_alphas[i] == 0:
                    choose_hat_i = 0
                else:
                    choose_hat_i = 1
                fXi = (self.hat_alphas - self.alphas) @ K_train[i, :] + self.b
                Ei = fXi - y_train[i]
                ### if statement checks if an example violates KKT conditions
                if (abs(Ei) < self.toler) or (self.alphas[i] == self.C) or (self.hat_alphas[i] == self.C):
                    continue
                else:
                    j, choose_hat_j = self.selectJrand(i, m)
                    fXj = (self.hat_alphas-self.alphas) @ K_train[j, :] + self.b
                    Ej = fXj - y_train[j]
                    alphaJold = self.alphas[j]
                    alphaIold = self.alphas[i]
                    hat_alphaJold = self.hat_alphas[j]
                    hat_alphaIold = self.hat_alphas[i]
                    if (choose_hat_i == 0) and (choose_hat_j == 1):
                        L = max(0, self.alphas[i] - self.hat_alphas[j])
                        H = min(self.C, self.C + self.alphas[i] - self.hat_alphas[j])
                    elif (choose_hat_i == 1) and (choose_hat_j == 0):
                        L = max(0, self.hat_alphas[i] - self.alphas[j])
                        H = min(self.C, self.C + self.hat_alphas[i] - self.alphas[j])
                    elif (choose_hat_i == 0) and (choose_hat_j == 0):
                        L = max(0, self.alphas[i] + self.alphas[j] - self.C)
                        H = min(self.C, self.alphas[i] + self.alphas[j])
                    else:
                        L = max(0, self.hat_alphas[i] + self.hat_alphas[j] - self.C)
                        H = min(self.C, self.hat_alphas[i] + self.hat_alphas[j])
                    if L == H:
                        continue
                    eta = 2.0 * K_train[j, i] - K_train[j, j] - K_train[i, i]
                    if eta >= 0:
                        continue
                    ### Update i 
                    if (choose_hat_i == 0) and (choose_hat_j == 1):
                        self.alphas[i] += (2 * self.toler - Ei + Ej) / eta
                    elif (choose_hat_i == 1) and (choose_hat_j == 0):
                        self.hat_alphas[i] += (2 * self.toler + Ei - Ej) / eta
                    elif (choose_hat_i == 0) and (choose_hat_j == 0):
                        self.alphas[i] += (-Ei + Ej) / eta
                    else:
                        self.hat_alphas[i] += (Ei - Ej) / eta
                    if choose_hat_i == 0:
                        self.alphas[i] = self.clipAlpha(self.alphas[i], H, L)
                    else:
                        self.hat_alphas[i] = self.clipAlpha(self.hat_alphas[i], H, L)
                    ### Whether the change amount is large enough or not
                    if choose_hat_i == 0:
                        if (abs(self.alphas[i] - alphaIold) < 0.00001):
                            continue
                    else:
                        if (abs(self.hat_alphas[i] - hat_alphaIold) < 0.00001):
                            continue
                    # update j by the same amount as i
                    if (choose_hat_i == 0) and (choose_hat_j == 1):
                        self.hat_alphas[j] += self.alphas[i] - alphaIold
                    elif (choose_hat_i == 1) and (choose_hat_j == 0):
                        self.alphas[j] += self.hat_alphas[i] - hat_alphaIold
                    elif (choose_hat_i == 0) and (choose_hat_j == 0):
                        self.alphas[j] += alphaIold - self.alphas[i]
                    else:
                        self.hat_alphas[j] += hat_alphaIold - self.hat_alphas[i]
                    # update b
                    b1 = self.b - Ej - (self.hat_alphas[j] - hat_alphaJold - (self.alphas[j] - alphaJold))    \
                        * K_train[j, j] - (self.hat_alphas[i] - hat_alphaIold - (self.alphas[i] - alphaIold)) * K_train[i, j]
                    b2 = self.b - Ei - (self.hat_alphas[j] - hat_alphaJold - (self.alphas[j] - alphaJold))   \
                        * K_train[j, i] - (self.hat_alphas[i] - hat_alphaIold - (self.alphas[i] - alphaIold)) * K_train[i, i]
                    if ((0 < self.alphas[j]) and (self.C > self.alphas[j])) or ((0 < self.hat_alphas[j]) and (self.C > self.hat_alphas[j])): 
                        self.b = b1
                    elif ((0 < self.alphas[i]) and (self.C > self.alphas[i])) or ((0 < self.hat_alphas[i]) and (self.C > self.hat_alphas[i])): 
                        self.b = b2
                    else: self.b = (b1 + b2) / 2.0
                    alphaPairsChanged += 1
            if (alphaPairsChanged == 0): 
                num_iter += 1
            else: 
                num_iter = 0
        return self.b, self.alphas, self.hat_alphas

    def predict(self, X_test):
        '''
        predict the label of test samples
        Args:
            X_test(matrix, num_test*num_features): features of test samples
        Returns:
            y_hat(vector, num_test): the predicted ouput of test samples
        '''
        K_test = self.rbfkernel(self.X_train, X_test)
        m = K_test.shape[0]
        y_hat = K_test.T @ (self.hat_alphas - self.alphas) + self.b
        return y_hat

## 1.3 Run SVR on the dataset

In [14]:
from sklearn.metrics import mean_squared_error
model_SVR = SVR(C = 0.5, sigma = 5, toler = 5, maxIter = 30)
b, alphas, hat_alphas = model_SVR.fit(X_train, y_train)
y_test_hat = model_SVR.predict(X_test)
mse = mean_squared_error(y_test_hat, y_test)
print("The MSE of SVR is:", mse)

The MSE of SVR is: 7796709.366960324
